In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
titles = []
years = []
time = []
imdb_ratings = []
metascore = []
votes = []
us_gross = []

In [11]:
base_url = "https://www.imdb.com"
second_url = "/search/title/?groups=top_1000"
headers = {"Accept-Language" : "en-US, en;q=0.5"}
i = 0

In [12]:
while True:
    print("Iteration", i+1)
    url = base_url + second_url
    results = requests.get(url, headers=headers)
    soup = BeautifulSoup(results.text, "html.parser") 

    movie_div = soup.find_all('div', class_='lister-item mode-advanced')    

    for container in movie_div:        
        #name
        name = container.h3.a.text
        titles.append(name)
        
        #year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        #runtime
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        #IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        #metascore
        meta = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascore.append(meta)
        
        #votes and grosses
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv)>1 else '-'
        us_gross.append(grosses)
        
    try:
        second_url = soup.find('a', class_='lister-page-next').get('href')
        i += 1
    except:
        print('No More url')
        break

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
No More url


In [13]:
movies = pd.DataFrame({
'movie':titles,
'year':years,
'timeMin':time,
'imdb':imdb_ratings,
'metascore':metascore,
'votes':votes,
'us_grossMillions':us_gross,
})

In [14]:
movies.dtypes

movie                object
year                 object
timeMin              object
imdb                float64
metascore            object
votes                object
us_grossMillions     object
dtype: object

In [15]:
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')
movies['votes'] = movies['votes'].str.replace(',','').astype(int)
movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')
movies.dtypes

movie                object
year                  int32
timeMin               int32
imdb                float64
metascore           float64
votes                 int32
us_grossMillions    float64
dtype: object

In [16]:
movies.to_csv('movies.csv')

In [18]:
movies.head()

,movie,year,timeMin,imdb,metascore,votes,us_grossMillions
0,Spider-Man: No Way Home,2021,148,8.7,71.0,411936,NaN
1,Dune,2021,155,8.1,74.0,457934,NaN
2,Harry Potter and the Sorcerer's Stone,2001,152,7.6,65.0,735516,317.58
3,West Side Story,2021,156,7.8,85.0,26526,NaN
4,The Gentlemen,2019,113,7.8,51.0,299637,NaN


In [19]:
url = base_url + second_url
results = requests.get(url, headers=headers)
# soup = BeautifulSoup(results.text, "html.parser")

In [25]:
files = open("text.html", 'w')
files.write(str(results.content))
files.close()
# print(results.text)